In [ ]:
import numpy as np
import cv2
import os

 
images_folder = './N2-adults-50um-SRx5-split-folders'
seg_folder = './N2-adults-50um-SRx5-seg-split-folders'

sub_folders = os.listdir(images_folder)

print(sub_folders)

crop_imags_folder = "N2-adults-50um-SRx5-crop-images-folders"
crop_seg_folder = "N2-adults-50um-SRx5-crop-seg-folders"


try:
    os.makedirs(crop_imags_folder)
    print(f"Folder '{crop_imags_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{crop_imags_folder}' already exists.")   


try:
    os.makedirs(crop_seg_folder)
    print(f"Folder '{crop_seg_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{crop_seg_folder}' already exists.")   


for sub_folder in sub_folders:

    sub_crop_imags_folder = crop_imags_folder + '/' + sub_folder
    sub_crop_segs_folder = crop_seg_folder + '/' + sub_folder
    try:
        os.makedirs(sub_crop_imags_folder)
        os.makedirs(sub_crop_segs_folder)
    except FileExistsError:
        pass

In [ ]:
for sub_folder in sub_folders:

    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)

    print(sub_folder)
    
    

    for image_name in images_list:
        
        if image_name.endswith('.png'):

            
            
            image_index = image_name.split('trimmed_')[1].split('_out')[0]


            origin_image = cv2.imread(images_list_dir + '/' + image_name)
            mask = cv2.imread(seg_images_list_dir + '/' + image_name)

            
            
            

            mask = mask[:,:,1] + mask[:,:,0]
            h,w = mask.shape

            mask[mask!=0] = 255

            remove_backgroud_image = np.zeros_like(origin_image)
            remove_backgroud_image = cv2.add(origin_image, np.zeros(np.shape(origin_image), dtype=np.uint8), mask=mask)

            
            
            
            images = np.zeros_like(mask)
            
            
            contours = cv2.findContours(
                mask.astype("uint8"), cv2.RETR_CCOMP,
                cv2.CHAIN_APPROX_NONE)[-2]

            
            


            cv2.drawContours(origin_image, contours[0], -1, (0, 0, 255), 1)


            

            c_index = 0

            

            for i in range(len(contours)):

                mu=cv2.moments(contours[i],False)
                center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                center_y = int(mu['m01'] / (mu['m00']+0.00000001))

                area = cv2.contourArea(contours[i])

                

                
                if area > 100000:
                    c_index = i
                
                
                


            

            rotRect = cv2.minAreaRect(contours[c_index]) 
            rect = cv2.boxPoints(rotRect).astype(np.int32)

            x,y,w,h = cv2.boundingRect(contours[c_index]) 

            center_x = int(x + w/2)
            center_y = int(y + h/2)

            
            
            
            extand_mask = np.zeros((3100,3280))
            extand_remove_backgroud_image = np.zeros((3100,3280,3))

            delta_h = int((3100 - mask.shape[0])/2)

            print(center_x,center_y,delta_h)

            center_y = center_y + delta_h

            
            extand_mask[delta_h:3100-delta_h,:] = mask
            extand_remove_backgroud_image[delta_h:3100-delta_h,:] = remove_backgroud_image

            
            

            
            

            extand_mask = extand_mask[center_y-1300:center_y+1300,center_x-240:center_x+240]
            extand_remove_backgroud_image = extand_remove_backgroud_image[center_y-1300:center_y+1300,center_x-240:center_x+240]

            cv2.imwrite(crop_imags_folder +'/'+ sub_folder+ '/' +str(image_index).zfill(3)+".png",extand_remove_backgroud_image)
            cv2.imwrite(crop_seg_folder   +'/'+ sub_folder+ '/' +str(image_index).zfill(3)+".png",extand_mask)

        
